In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('finalDataset.csv')
df

,id,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year
0,7lmeHLHBe4nmXzuXc0HDjk,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999
1,1wsRitfRRtWyEapl0q22o8,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999
2,1hR0fIFK2qRG3f3RF70pb7,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999
3,2lbASgTSoDO7MTuLAXlTW0,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999
4,1MQTmpYOZ6fcMQc56Hdo7T,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204007,0EsMifwUmMfJZxzoMPXJKZ,0,0.264,0.966,5,-6.970,0,0.0672,0.00935,0.002240,0.3370,0.415,159.586,276213,4.0,2014
1204008,2WSc2TB1CSJgGE0PEzVeiu,0,0.796,0.701,11,-6.602,0,0.0883,0.10400,0.644000,0.0749,0.781,121.980,363179,4.0,2014
1204009,6iProIgUe3ETpO6UT0v5Hg,0,0.785,0.796,9,-5.960,0,0.0564,0.03040,0.918000,0.0664,0.467,121.996,385335,4.0,2014
1204010,37B4SXC8uoBsUyKCWnhPfX,0,0.665,0.856,6,-6.788,0,0.0409,0.00007,0.776000,0.1170,0.227,124.986,324455,4.0,2014


In [3]:
df.columns

Index(['id', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'year'],
      dtype='object')

K - Nearest Neighbours algorithm

In [4]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import joblib

features = ['explicit', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'year']

# Extracting features and song IDs
X = df[features]
song_ids = df['id']

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize the Nearest Neighbors model
k_neighbors = 15  # the number of neighbors 
model = NearestNeighbors(n_neighbors=k_neighbors, metric='euclidean')
model.fit(X_scaled)

# Save the trained model using joblib
model_filename = 'knn_model.joblib'
joblib.dump(model, model_filename)

['knn_model.joblib']

In [5]:
allFeaturedDataset = pd.read_csv('tracks_features.csv')

# function to recommend song based on a given song ID and output its name and artist name from the ID:
def recommendSong(songId):
    # get the index of the song ID
    song_index = df[df['id'] == songId].values[0]
    
    query_song_features = X_scaled[song_index].reshape(1, -1)
    
    # Find the k nearest neighbours of the queried song
    distances, indices = model.kneighbors(query_song_features)
    
    # Get recommended song IDs
    recommended_song_ids = song_ids.iloc[indices[0]].values.tolist()
    
    # Get the names of the recommended songs
    recommendations = allFeaturedDataset[allFeaturedDataset['id'].isin(recommended_song_ids)][['name', 'artists']]
 
    # sort the dataframe by distance
    recommendations = recommendations.sort_values(by='distance', ascending=True)
    # return the dataframe
    return recommendations

In [ ]:
input_song_id = '7lmeHLHBe4nmXzuXc0HDjk' 
recommended_songs = recommendSong(input_song_id)

# Display the recommended songs
print("Recommended Songs:")
print(recommended_songs)